# Identificando elementos que não sejam o Campo


Para fazer identificação dos elementos pertencentes ao campo foi desconsiderado todos os elementos pixels que nem fossem verdes

In [110]:
import cv2
import os
import numpy as np

vidcap = cv2.VideoCapture('../videos/video6.mp4')
success,image = vidcap.read()
success = True

while success:

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    res = cv2.bitwise_and(image, image, mask=mask)
    cv2.imshow('partida',res)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success,image = vidcap.read()
    
vidcap.release()
cv2.destroyAllWindows()

# Identificando jogadores do time 

* Para idenficar os jogadores de cada time foi considerado a cor da blusa
* Foi removido do e setado como preto e contado o numero do pixels pretos para fazer a identificação

In [1]:
import cv2
import os
import numpy as np

vidcap = cv2.VideoCapture('../videos/video6.mp4')
success,image = vidcap.read()
success = True

upper_green = np.array([70,255, 255])
lower_green = np.array([40,40, 40])

lower_blue = np.array([110,50,50])
upper_blue = np.array([130,255,255])

lower_red = np.array([0,31,255])
upper_red = np.array([176,255,255])

lower_white = np.array([0,0,0])
upper_white = np.array([0,0,255])

lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])

lower_black = np.array([20, 100, 100])
upper_black = np.array([180, 255, 30])


def detect_player(player_img, low_color, upper_color):
    player_hsv = cv2.cvtColor(player_img,cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(player_hsv, low_color, upper_color)
    res = cv2.bitwise_and(player_img, player_img, mask=mask)
    res = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    nzCount = cv2.countNonZero(res)
    return nzCount

while success:
    time.sleep(0.01)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    res = cv2.bitwise_and(image, image, mask=mask)
    res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(res_gray ,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if(h>=(1.5)*w):
            if(w>15 and h>= 15):
                player_img = image[y:y+h,x:x+w]
                defensor = detect_player(player_img, lower_red, upper_red)
                atacante = detect_player(player_img, lower_white, upper_white)
                if(defensor >= 20):
                    cv2.putText(image, 'Defensor', (x-2, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2, cv2.LINE_AA)
                    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)
                if(atacante>=20):
                    cv2.putText(image, 'Atacante', (x-2, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
                    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)

    cv2.imshow('partida',image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success,image = vidcap.read()
    
vidcap.release()
cv2.destroyAllWindows()

# Identificando Jogadores sem Uso de cores estáticas

* Baseado no histograma

In [11]:
import cv2
import os
import numpy as np
import time

vidcap = cv2.VideoCapture('../videos/video9.mp4')
success,image = vidcap.read()
success = True

lower = np.array([])
upper = np.array([])

CURRENT = None

while success:
    time.sleep(0.01)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_green, upper_green)
    res = cv2.bitwise_and(image, image, mask=mask)
    res_gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    
    contours, _ = cv2.findContours(res_gray ,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        if(h>=(1.5)*w):
            if(w>15 and h>= 15):
                player_img = image[y:y+h,x:x+w]
                if(CURRENT is None):
                    hist = cv2.calcHist([player_img],[0],None,[256],[0,256])
                    CURRENT = hist
                if(CURRENT is not None):
                    hist = cv2.calcHist([player_img],[0],None,[256],[0,256])
                    result = cv2.compareHist( hist, CURRENT, cv2.HISTCMP_BHATTACHARYYA)
                    CURRENT = hist
                if(result < 0.49):
                    cv2.putText(image, 'Defensor', (x-2, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2, cv2.LINE_AA)
                    cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),3)
                else:
                    cv2.putText(image, 'Atacante', (x-2, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2, cv2.LINE_AA)
                    cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),3)
       
    cv2.imshow('partida',image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    success,image = vidcap.read()
    
vidcap.release()
cv2.destroyAllWindows()
